In [2]:
# Transformers installation
! pip install transformers datasets evaluate seqeval accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
#Loading Data
import json
with open("processed/main.json", encoding='utf-8') as fp:
    dataset = json.load(fp);

'''for i in range(len(dataset)):
    for j in range(len(dataset[i]['tokens'])):
        dataset[i]['tokens'][j] = dataset[i]['tokens'][j].upper()'''
dataset[0]

{'id': '0',
 'ner_tags': ['B-no', 'B-method', 'B-no', 'B-method', 'B-class', 'B-no'],
 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}

In [5]:

label2id = {
    "O":0,
    "B-no":1,
    "I-no":2,
    "B-class":3,
    "I-class":4,
    "B-method":5,
    "I-method":6,
    "B-attribute":7,
    "I-attribute":8,
    "B-association":9,
    "I-association":10,
    "B-generalization":11,
    "I-generalization":12,
}

id2label = {
    0:"O",
    1:"B-no",
    2:"I-no",
    3:"B-class",
    4:"I-class",
    5:"B-method",
    6:"I-method",
    7:"B-attribute",
    8:"I-attribute",
    9:"B-association",
    10:"I-association",
    11:"B-generalization",
    12:"I-generalization",
}

In [6]:
for i in range(len(dataset)):
    dataset[i]["ner_tags"] = [label2id[j] for j in dataset[i]["ner_tags"]]

dataset[:10]

[{'id': '0',
  'ner_tags': [1, 5, 1, 5, 3, 1],
  'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']},
 {'id': '1', 'ner_tags': [1, 2], 'tokens': ['3.1.1.1', 'Brief Description']},
 {'id': '2',
  'ner_tags': [1,
   2,
   2,
   2,
   3,
   1,
   2,
   2,
   9,
   1,
   9,
   3,
   1,
   0,
   1,
   3,
   1,
   2,
   2,
   2,
   2,
   3,
   0,
   1,
   2,
   0],
  'tokens': ['This',
   'use case',
   'provides',
   'the',
   'actor',
   'the',
   'means',
   'to',
   'start',
   'and',
   'stop',
   'diagnostic',
   'processing',
   '.',
   'The',
   'actor',
   'in',
   'this',
   'case',
   'is',
   'the',
   'administrator user',
   '(',
   'see',
   '2.5.1',
   ').']},
 {'id': '3',
  'ner_tags': [1, 2, 2, 2],
  'tokens': ['3.1.1.2', 'Flow', 'of', 'Events']},
 {'id': '4',
  'ner_tags': [1, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 0],
  'tokens': ['The',
   'use case',
   'begins',
   'just',
   'prior',
   'to',
   'when',
   'the',
   'user',
   'launches',
   'the',
   'a

In [7]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset,test_size=0.15)
print(len(train_set),len(test_set))

3725 658


In [8]:
from datasets import IterableDataset

def gen_train():
    for i in train_set:
        yield i

def gen_test():
    for i in test_set:
        yield i

train_dataset = IterableDataset.from_generator(gen_train)
test_dataset = IterableDataset.from_generator(gen_test)

In [9]:
label_list = list(label2id.keys())
label_list

['O',
 'B-no',
 'I-no',
 'B-class',
 'I-class',
 'B-method',
 'I-method',
 'B-attribute',
 'I-attribute',
 'B-association',
 'I-association',
 'B-generalization',
 'I-generalization']

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [11]:
# example = wnut["train"][0]
example = dataset[0]
print(example)
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

{'id': '0', 'ner_tags': [1, 5, 1, 5, 3, 1], 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}


['[CLS]',
 '3',
 '.',
 '1',
 '.',
 '1',
 'start',
 'and',
 'stop',
 'diagnostic',
 '##s',
 'use',
 'case',
 '[SEP]']

In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
# tokenized_dataset = list(map(tokenize_and_align_labels, dataset))
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

In [14]:
for i in tokenized_test_dataset:
    print(i)

{'id': '4017', 'ner_tags': [1, 3, 1, 3, 1, 5, 1, 7, 3, 1, 3, 0], 'tokens': ['The', 'deputyLibrarian', 'is', 'incharge', 'of', 'receiving', 'the', 'Returned', 'Books', 'and', 'Journals', '.'], 'input_ids': [101, 1996, 4112, 29521, 19848, 2937, 2003, 4960, 2906, 3351, 1997, 4909, 1996, 2513, 2808, 1998, 9263, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 1, 3, -100, -100, -100, 1, 3, -100, -100, 1, 5, 1, 7, 3, 1, 3, 0, -100]}
{'id': '1251', 'ner_tags': [0, 1, 0, 3, 1, 3, 0, 3, 0, 3, 1, 3], 'tokens': ['•', 'Actors', ':', 'VUB-network users', 'and', 'Members', ',', 'Publishers', ',', 'Moderators', 'and', 'Administrators'], 'input_ids': [101, 1528, 5889, 1024, 24728, 2497, 1011, 2897, 5198, 1998, 2372, 1010, 8544, 1010, 29420, 2015, 1998, 15631, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 1, 0, 3, -100, -100, -100, -100, 1, 3, 0, 3, 0, 3, -100, 1, 3, -100]}
{'id': '1243', '

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
import evaluate

seqeval = evaluate.load("seqeval")

In [17]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [18]:
# done before

In [19]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [20]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_steps = 30,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


max_steps is given, it will override any value given in num_train_epochs


In [21]:
trainer.train()

/home/codespace/.python/current/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 960
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 30
The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__`

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.527060,0.000000,0.000000,0.000000,0.538207


***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/codespace/.python/current/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.python/current/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Sav

TrainOutput(global_step=30, training_loss=2.0089847564697267, metrics={'train_runtime': 697.626, 'train_samples_per_second': 1.376, 'train_steps_per_second': 0.043, 'total_flos': 27148580133120.0, 'train_loss': 2.0089847564697267, 'epoch': 1.0})

In [22]:
# trainer.push_to_hub()
#trainer.save_model("/content/v1/")

In [23]:
#!zip -r v1.zip v1/

In [29]:
text = "Some of the big cities may have more than one airports."

In [30]:
from transformers import pipeline

classifier = pipeline("ner", model=trainer.model, tokenizer=trainer.tokenizer)
classifier(text)

[]

In [26]:
'''from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")
inputs = tokenizer(text, return_tensors="pt")'''

'from transformers import AutoTokenizer\n\ntokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")\ninputs = tokenizer(text, return_tensors="pt")'

In [27]:
'''from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")
with torch.no_grad():
    logits = model(**inputs).logits'''

'from transformers import AutoModelForTokenClassification\n\nmodel = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")\nwith torch.no_grad():\n    logits = model(**inputs).logits'

In [28]:
'''predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class'''

'predictions = torch.argmax(logits, dim=2)\npredicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]\npredicted_token_class'